In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

In [3]:
raw_dataset = pd.read_csv(url, sep=' ', names=column_names, na_values='?', comment='\t', skipinitialspace=True)

In [4]:
dataset = raw_dataset.copy()
dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [5]:
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [6]:
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

In [7]:
dataset = dataset.dropna()

In [8]:
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      0
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

In [9]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [10]:
dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA


In [11]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')

In [12]:
dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
0,18.0,8,307.0,130.0,3504.0,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,0,0,1


In [13]:
train_data = dataset.sample(frac=0.8, random_state=0)
test_data = dataset.drop(train_data.index)

In [14]:
train_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
MPG,314.0,23.310510,7.728652,10.0,17.00,22.0,28.95,46.6
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0


In [15]:
train_features = train_data.copy()
test_features = test_data.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

In [16]:
from tensorflow.keras.layers.experimental import preprocessing

normalizer = preprocessing.Normalization()

In [17]:
import numpy as np
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

[5.4777069e+00 1.9531847e+02 1.0486943e+02 2.9902517e+03 1.5559236e+01
 7.5898087e+01 1.7834395e-01 1.9745223e-01 6.2420380e-01]


In [18]:
acceleration = np.array(train_features['Acceleration'])
print(acceleration[:3])
acceleration_normalizer = preprocessing.Normalization(input_shape=[1,])
acceleration_normalizer.adapt(acceleration)
acceleration_normalizer.mean.numpy()

[14.5 17.3 13.5]


array([15.559236], dtype=float32)

In [19]:
acceleration_model = tf.keras.Sequential([
    acceleration_normalizer,
    tf.keras.layers.Dense(units=1)
])

In [20]:
acceleration_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 1)                 3         
_________________________________________________________________
dense (Dense)                (None, 1)                 2         
Total params: 5
Trainable params: 2
Non-trainable params: 3
_________________________________________________________________


In [21]:
acceleration_model.predict(acceleration[:10])

array([[ 0.3305753 ],
       [-0.54327244],
       [ 0.64266384],
       [ 0.26815766],
       [-0.76173466],
       [-0.51206404],
       [-1.0114052 ],
       [-1.9476708 ],
       [-0.2936018 ],
       [-1.0738232 ]], dtype=float32)

In [22]:
acceleration_model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.1), loss='mae',)

In [23]:
acceleration_model.fit(acceleration, train_labels, epochs=100, validation_split=0.2)

Epoch 1/100
8/8 [==============================] - 1s 91ms/step - loss: 23.0069 - val_loss: 22.9890
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 21.7843 - val_loss: 22.1934
Epoch 3/100
8/8 [==============================] - 0s 10ms/step - loss: 21.0486 - val_loss: 21.4023
Epoch 4/100
8/8 [==============================] - 0s 9ms/step - loss: 21.0821 - val_loss: 20.6072
Epoch 5/100
8/8 [==============================] - 0s 9ms/step - loss: 19.3585 - val_loss: 19.8215
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: 19.0659 - val_loss: 19.0308
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 18.7566 - val_loss: 18.2335
Epoch 8/100
8/8 [==============================] - 0s 11ms/step - loss: 17.1960 - val_loss: 17.4337
Epoch 9/100
8/8 [==============================] - 0s 9ms/step - loss: 17.1782 - val_loss: 16.6395
Epoch 10/100
8/8 [==============================] - 0s 8ms/step - loss: 15.6259 - val_loss: 15.8465
Epoch

In [24]:
test_data.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
9,15.0,8,390.0,190.0,3850.0,8.5,70,0,0,1
25,10.0,8,360.0,215.0,4615.0,14.0,70,0,0,1
28,9.0,8,304.0,193.0,4732.0,18.5,70,0,0,1
31,25.0,4,113.0,95.0,2228.0,14.0,71,0,1,0
33,19.0,6,232.0,100.0,2634.0,13.0,71,0,0,1


In [25]:
acceleration_model.evaluate(test_data['Acceleration'], test_labels)

3/3 [==============================] - 0s 3ms/step - loss: 5.7316


5.731557369232178

In [26]:
# Linear Regression
linear_model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(1)
])

In [27]:
linear_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='mae')

In [28]:
linear_model.fit(train_features, train_labels, epochs=100, validation_split=0.2)

Epoch 1/100
8/8 [==============================] - 1s 28ms/step - loss: 23.3390 - val_loss: 22.7200
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 22.2660 - val_loss: 22.0078
Epoch 3/100
8/8 [==============================] - 0s 11ms/step - loss: 21.3151 - val_loss: 21.2846
Epoch 4/100
8/8 [==============================] - 0s 14ms/step - loss: 20.2820 - val_loss: 20.5647
Epoch 5/100
8/8 [==============================] - 0s 14ms/step - loss: 19.8815 - val_loss: 19.8155
Epoch 6/100
8/8 [==============================] - 0s 11ms/step - loss: 18.9470 - val_loss: 19.0932
Epoch 7/100
8/8 [==============================] - 0s 19ms/step - loss: 17.8632 - val_loss: 18.3855
Epoch 8/100
8/8 [==============================] - 0s 18ms/step - loss: 17.1881 - val_loss: 17.6210
Epoch 9/100
8/8 [==============================] - 0s 22ms/step - loss: 16.7342 - val_loss: 16.8902
Epoch 10/100
8/8 [==============================] - 0s 13ms/step - loss: 15.4215 - val_loss: 16.1754

In [29]:
linear_model.evaluate(test_features, test_labels)

3/3 [==============================] - 0s 3ms/step - loss: 2.5726


2.5725760459899902

In [36]:
def build_and_compile_model(norm):
    model = tf.keras.Sequential([
        norm,
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='mae')
    return model

In [37]:
dnn = build_and_compile_model(normalizer)

In [38]:
dnn.fit(train_features, train_labels, epochs=100, validation_split=0.2)

Epoch 1/100
8/8 [==============================] - 1s 31ms/step - loss: 17.3596 - val_loss: 15.3922
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 10.9098 - val_loss: 6.7102
Epoch 3/100
8/8 [==============================] - 0s 13ms/step - loss: 5.3892 - val_loss: 3.3032
Epoch 4/100
8/8 [==============================] - 0s 11ms/step - loss: 4.6686 - val_loss: 4.6193
Epoch 5/100
8/8 [==============================] - 0s 11ms/step - loss: 3.6514 - val_loss: 3.4842
Epoch 6/100
8/8 [==============================] - 0s 11ms/step - loss: 2.8261 - val_loss: 2.5001
Epoch 7/100
8/8 [==============================] - 0s 12ms/step - loss: 2.1434 - val_loss: 2.7913
Epoch 8/100
8/8 [==============================] - 0s 10ms/step - loss: 2.5093 - val_loss: 2.3137
Epoch 9/100
8/8 [==============================] - 0s 10ms/step - loss: 2.0185 - val_loss: 2.0937
Epoch 10/100
8/8 [==============================] - 0s 12ms/step - loss: 2.0914 - val_loss: 2.2526
Epoch 11/100
8/8

In [39]:
dnn.evaluate(test_features, test_labels)

3/3 [==============================] - 0s 6ms/step - loss: 1.8848


1.8848083019256592